In [32]:
# 使用 PyMySQL 來將下載讀取完成的資料寫入 MySQL 資料庫保存，並且再讀取出來



# 1. 先將資料下載並做成 csv 格式資料

import requests
import io
import csv

response = requests.get("https://data.kcg.gov.tw/File/DirectDownload/d8e11066-81e7-44ee-963d-4fdc37055cfc")

io_file = io.StringIO(response.content.decode())

csv_file = list(csv.reader(io_file))

data_column , *data_row = csv_file

# print(data_column)

# for row in data_row : 

#     print(row)



# 2. 再使用 PyMySQL 對資料庫做存取和異動 ( Insert , Update , Delete )

import pymysql

# 記得就是先建立連線 connection，再建立指標 cursor

connection = pymysql.connect(

    host="127.0.0.1",
    user="root",
    password="12345",
    database="example",
    charset="utf8",
    autocommit=False

)

cursor = connection.cursor()

# 就是打以前 SQL 語言用的一樣增刪修指令

sql = "INSERT INTO new_table (Seq,年度,家數,實際住用客房數,可供住用客房數,住用率) VALUES (%s,%s,%s,%s,%s,%s)"

# 再做 execute 移動指標並確認異動 commit

try : 

    result = cursor.executemany(sql,data_row)

    connection.commit()

except Exception as e : 

    connection.rollback()

    print("變動失敗",e)

    cursor.close()

    connection.close()

# 記得要把連線關掉，不然資料庫會不能做任何執行動作

cursor.close()

connection.close()



# 3. 再使用 PyMySQL 來讀取資料庫和資料 ( Select )

# 一樣都先從建立 connection 和 cursor 開始

connection = pymysql.connect(

    host="localhost",
    user="root",
    password="12345",
    database="example",
    charset="utf8"

)

cursor = connection.cursor()

# 打 SQL 語言的查詢命令 ( 用之前得到的筆數來查詢最後幾筆是不是那些資料 )

sql = f"SELECT * FROM new_table ORDER BY Seq DESC LIMIT {result}"

# 再做 execute 和抓資料 fetch 

cursor.execute(sql)

results = cursor.fetchall()

for result in results : 

    print(result)

cursor.close()

connection.close()





ConnectionError: HTTPSConnectionPool(host='data.kcg.gov.tw', port=443): Max retries exceeded with url: /File/DirectDownload/d8e11066-81e7-44ee-963d-4fdc37055cfc (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x0000015F9B3E82D0>: Failed to resolve 'data.kcg.gov.tw' ([Errno 11001] getaddrinfo failed)"))

In [2]:
# 不一樣的版本練習，增刪修和查詢都改用逐筆的方式



import requests
import io
import csv

response = requests.get("https://data.kcg.gov.tw/File/DirectDownload/32a0f3d2-f23f-4333-a776-135e518bd861")

file = response.content.decode("utf-8-sig")

csv_file = list(csv.reader(io.StringIO(file)))

data_column , *data_row = csv_file



import pymysql

# 1. 用逐行方式 + for 迴圈寫入 ( execute vs executemany)

connection = pymysql.connect(

    host="127.0.0.1",
    user="root",
    database="example",
    password="12345",
    charset="utf8"

)

cursor = connection.cursor()

sql = "INSERT INTO new_table_2 (Seq,年度,國小補校校數,國中補校校數,高級中等學校進修部校數,國小補校班級數,國中補校班級數,高級中等學校進修部班級數) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"

for item in data_row : 

    cursor.execute(sql,item)

connection.commit()

cursor.close()

connection.close()



# 2. 用逐行方式 + while 迴圈讀取 ( cursor vs SScursor )

connection = pymysql.connect(

    host="127.0.0.1",
    user="root",
    database="example",
    password="12345",
    charset="utf8"

)

cursor = connection.cursor(pymysql.cursors.SSCursor)

sql = "SELECT * FROM new_table_2"

cursor.execute(sql)

while True : 

    data = cursor.fetchone()

    if data is None : 

        break

    print(data)



('1', '2017', '15', '24', '17', '23', '44', '172')
('2', '2018', '15', '23', '17', '22', '43', '165')
('3', '2019', '16', '22', '17', '24', '36', '155')
('4', '2020', '15', '20', '15', '23', '35', '133')
('5', '2021', '15', '17', '15', '23', '32', '128')
('6', '2022', '14', '16', '14', '20', '29', '120')
('7', '2023', '13', '16', '14', '20', '29', '114')
('8', '2024', '13', '16', '14', '18', '28', '105')
